In [2]:
import torch
import torch.nn as nn
import numpy as np
import matplotlib.pyplot as plt
import time



In [ ]:
class RC(nn.Module):
    def __init__(self) -> None:
        super(RC, self).__init__()
        
    def forward(self, input):
        pass